<img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to troubleshoot your EC2 configuration in a private subnet using unSkript legos.</b>
</div>

<br>

<center><h2>Troubleshooting Your EC2 Configuration in a Private Subnet</h2></center>

# Steps Overview
    1) Get all details of given instance id and capture the vpc id.
    2) Using VPC ID get the NAT Gateway details.
    3) Using VPC ID get the Internet Gateway details.
    3) SSH given instance and try to connect to the internet.

Here we will use unSkript Get AWS Instance Details Lego. This lego takes instance_id: str and region: str as input. This input is used to discover all details about given EC2 instance.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Dict
import pprint


from beartype import beartype
@beartype
def aws_get_instance_details(
    handle,
    instance_id: str,
    region: str,
) -> Dict:

    ec2client = handle.client('ec2', region_name=region)
    instances = []
    response = ec2client.describe_instances(
        Filters=[{"Name": "instance-id", "Values": [instance_id]}])
    for reservation in response["Reservations"]:
        for instance in reservation["Instances"]:
            instances.append(instance)
    return instances[0]


def unskript_default_printer(output):
    if isinstance(output, (list, tuple)):
        for item in output:
            print(f'item: {item}')
    elif isinstance(output, dict):
        for item in output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(output)

task = Task(Workflow())
task.configure(inputParamsJson='''{
    "instance_id": "Instance_id",
    "region": "Region"
    }''')
task.configure(outputName="IncidentDetails")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_instance_details, lego_printer=unskript_default_printer, hdl=hdl, args=args)

Here we will use unSkript AWS Get NAT Gateway Info by VPC ID Lego. This lego takes vpc_id: str, region: str as input. This input is used to discover details about NAT gateway by using VPC ID.

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype
@beartype
def aws_get_natgateway_by_vpc_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_natgateway_by_vpc(handle, vpc_id: str, region: str) -> Dict:
    """aws_get_natgateway_by_vpc Returns an Dict of NAT Gateway info.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type vpc_id: str
        :param vpc_id: VPC ID to find NAT Gateway.

        :type region: str
        :param region: Region to filter instance.

        :rtype: Dict of NAT Gateway info.
    """

    ec2Client = handle.client('ec2', region_name=region)
    result = {}
    try:
        response = aws_get_paginator(ec2Client, "describe_nat_gateways", "NatGateways",
                                Filters=[{'Name': 'vpc-id','Values': [vpc_id]}])
        for nat_info in response:
            if "NatGatewayId" in nat_info:
                result["NatGatewayId"] = nat_info["NatGatewayId"]
            if "State" in nat_info:
                result["State"] = nat_info["State"]
            if "SubnetId" in nat_info:
                result["SubnetId"] = nat_info["SubnetId"]
    except Exception as error:
        result["error"] = error

    return result




task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "Region",
    "vpc_id": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "[n['VpcId'] for n in IncidentDetails.get('NetworkInterfaces') if n.get('VpcId') != None ]",
    "iter_parameter": "vpc_id"
    }''')
task.configure(outputName="NATOutput")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_natgateway_by_vpc, lego_printer=aws_get_natgateway_by_vpc_printer, hdl=hdl, args=args)

In [ ]:
import pprint
pprint.pprint(NATOutput)

Here we will use unSkript AWS Get Internet Gateway by VPC ID Lego. This lego takes vpc_id: str, region: str as input. This input is used to discover name of Internet gateway by using VPC ID.

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint

from beartype import beartype
@beartype
def aws_get_internet_gateway_by_vpc_printer(output):
    if output is None:
        return
    pprint.pprint(output)

@beartype
def aws_get_internet_gateway_by_vpc(handle, vpc_id: str, region: str) -> List:
    """aws_get_internet_gateway_by_vpc Returns an List of internet Gateway.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type vpc_id: str
        :param vpc_id: VPC ID to find NAT Gateway.

        :type region: str
        :param region: Region to filter instance.

        :rtype: List of Internet Gateway.
    """

    ec2Client = handle.client('ec2', region_name=region)
    result = []
    try:
        response = aws_get_paginator(ec2Client, "describe_internet_gateways", "InternetGateways",
                                Filters=[{'Name': 'attachment.vpc-id','Values': [vpc_id]}])
        for nat_info in response:
            if "InternetGatewayId" in nat_info:
                result.append(nat_info["InternetGatewayId"])

    except Exception as error:
        result.append({"error":error})

    return result






task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "Region",
    "vpc_id": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "[n['VpcId'] for n in IncidentDetails.get('NetworkInterfaces') if n.get('VpcId') != None ]",
    "iter_parameter": "vpc_id"
    }''')
task.configure(outputName="InternetOutput")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_internet_gateway_by_vpc, lego_printer=aws_get_internet_gateway_by_vpc_printer, hdl=hdl, args=args)

In [ ]:
import pprint
pprint.pprint(InternetOutput)

Here we will use unSkript SSH Execute Remote Command Lego. This lego takes hosts: List[str], command: str, sudo: bool as input. This inputs is used to connect to instance and check weather the the instance connected to internet.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
import pprint


from beartype import beartype
@beartype
def ssh_execute_remote_command_printer(output):
    if output is None:
        return
    print("\n")
    pprint.pprint(output)


@beartype
def ssh_execute_remote_command(sshClient, hosts: List[str], command: str, sudo: bool = False) -> Dict:

    client = sshClient(hosts)
    runCommandOutput = client.run_command(command=command, sudo=sudo)
    client.join()
    res = {}

    for host_output in runCommandOutput:
        hostname = host_output.host
        output = []
        for line in host_output.stdout:
            output.append(line)

        o = "\n".join(output)
        res[hostname] = o

    return res


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "command": "Command",
    "hosts": "[n['PrivateIpAddress'] for n in IncidentDetails.get('NetworkInterfaces') if n.get('PrivateIpAddress') != None ]",
    "sudo": "False"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(ssh_execute_remote_command, lego_printer=ssh_execute_remote_command_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to perform AWS action and this runbook collect the instance details and then by using vpc id it will check for NAT Gateway and Internet Gateway. To view the full platform capabilities of unSkript please visit https://unskript.com